In [ ]:
#hide   
import warnings
import gc
from tqdm.notebook import tqdm 
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
from datetime import datetime as dt
from typing import *
import rfpy
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from rfpy.main import export_metadata
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import *
import numpy as np
import pandas as pd

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 

In [ ]:
TH = -80
def identify_peaks(df, th=TH, occ=50):
    dict_freq = {}
    for c in tqdm(df.columns):
        d = df.loc[:, c].value_counts().to_dict()
        d = {k:v for k,v in d.items() if k > th}
        if d:
            dict_freq[c] = d
    counts = [(k,sum(v.values())/df.shape[0]*100) for k,v in dict_freq.items()]
    counts.sort(key=lambda x: x[1], reverse=True)
    return [(i,f'{j:.2f}%') for (i,j) in counts if j >= occ]

Caminho das pastas de entrada e saída

In [ ]:
gru = Path(r'D:/OneDrive - ANATEL/Backup_Rfeye_SP/GRU/2021/rfeye002239_210313_T210520_MaskBroken.bin')
cpv = Path(r'D:\OneDrive - ANATEL\Backup_Rfeye_SP\CPV\2021\rfeye002310_210204_T184431_MaskBroken.bin')
rpo = Path(r'D:\OneDrive - ANATEL\Backup_Rfeye_SP\RPO\2021\rfeye002304_210416_T062340_MaskBroken.bin')
sjrp = Path(r'D:\OneDrive - ANATEL\Backup_Rfeye_SP\SJRP\2021\rfeye002266_210415_T113755_MaskBroken.bin')

Função que mapeia os arquivos da pasta e retorna um lista generalizada `L`

In [ ]:
gru = parse_bin(gru)
cpv = parse_bin(cpv)
rpo = parse_bin(rpo)
sjrp = parse_bin(sjrp)

TypeError: expected str, bytes or os.PathLike object, not dict

In [ ]:
blocos = [(k, len(v)) for k,v in rpo['blocks'].items()]
blocos.sort(key=lambda x: x[1], reverse=True)
blocos

[((68, 301), 2975),
 ((40, 1), 254),
 ((68, 311), 244),
 ((68, 321), 51),
 ((68, 371), 45),
 ((68, 391), 44),
 ((68, 341), 14),
 ((68, 361), 10),
 ((42, 1), 4),
 ((42, 0), 2),
 ((21, 0), 1),
 ((42, 301), 1),
 ((42, 311), 1),
 ((42, 321), 1),
 ((42, 331), 1),
 ((42, 341), 1),
 ((42, 351), 1),
 ((42, 361), 1),
 ((42, 371), 1),
 ((42, 381), 1),
 ((42, 391), 1)]

In [ ]:
%%time
c = rpo['blocks'][(68,301)]
df = extract_levels(c)
df

Wall time: 226 ms


,108.000000,108.001953,108.003907,108.005860,108.007813,108.009766,108.011720,108.013673,108.015626,108.017579,...,136.982421,136.984374,136.986327,136.988280,136.990234,136.992187,136.994140,136.996093,136.998047,137.000000
0,-98.0,-93.5,-95.0,-99.5,-99.5,-99.5,-93.5,-89.0,-89.0,-93.5,...,-101.0,-101.0,-101.0,-101.0,-99.5,-99.5,-101.0,-101.0,-99.5,-99.0
1,-100.0,-99.5,-101.0,-101.0,-101.0,-99.5,-94.0,-96.5,-99.5,-100.0,...,-101.0,-101.0,-101.0,-99.5,-97.0,-98.0,-109.5,-101.0,-101.0,-101.0
2,-101.0,-101.0,-101.0,-101.0,-97.0,-98.0,-95.0,-93.5,-93.5,-93.0,...,-98.5,-99.0,-104.5,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
3,-96.5,-95.5,-98.5,-98.5,-98.5,-91.5,-91.0,-93.5,-97.0,-94.5,...,-97.5,-98.0,-101.0,-101.0,-97.5,-99.5,-105.0,-101.0,-101.0,-101.0
4,-96.5,-98.5,-100.0,-98.5,-101.0,-101.0,-97.5,-94.0,-92.5,-94.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,-101.0,-101.0,-101.0,-93.5,-96.5,-97.5,-97.5,-101.0,-101.0,-95.5,...,-99.5,-95.0,-93.0,-94.5,-98.0,-99.5,-103.5,-101.0,-101.0,-101.0
2971,-97.5,-101.0,-101.0,-94.5,-93.0,-97.5,-100.0,-97.0,-100.0,-95.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
2972,-99.5,-101.0,-101.0,-98.0,-99.5,-101.0,-101.0,-101.0,-98.5,-99.0,...,-97.0,-100.0,-98.5,-97.5,-96.5,-97.0,-105.0,-101.0,-101.0,-101.0
2973,-95.5,-96.0,-97.0,-91.5,-99.5,-96.0,-92.0,-93.5,-98.5,-95.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-97.5,-107.0,-101.0,-101.0,-101.0


In [ ]:
identify_peaks(df, occ=10)

  0%|          | 0/14848 [00:00<?, ?it/s]

[(127.42318313464, '97.51%'),
 (127.42513639119014, '77.98%'),
 (127.42122987808985, '29.01%'),
 (119.99885498753957, '17.01%'),
 (119.54765272445611, '16.20%'),
 (119.54960598100627, '16.20%')]

In [ ]:
blocos = [(k, len(v)) for k,v in sjrp['blocks'].items()]
blocos.sort(key=lambda x: x[1], reverse=True)
blocos

[((68, 301), 16243),
 ((40, 1), 1372),
 ((68, 311), 659),
 ((68, 381), 179),
 ((68, 341), 41),
 ((42, 1), 22),
 ((68, 361), 10),
 ((42, 0), 2),
 ((21, 0), 1),
 ((42, 301), 1),
 ((42, 311), 1),
 ((42, 321), 1),
 ((42, 331), 1),
 ((42, 341), 1),
 ((42, 351), 1),
 ((42, 361), 1),
 ((42, 371), 1),
 ((42, 381), 1),
 ((42, 391), 1),
 ((68, 321), 1)]

In [ ]:
%%time
c = sjrp['blocks'][(68,301)]
df = extract_levels(c)
df

Wall time: 1.11 s


,108.000000,108.001953,108.003907,108.005860,108.007813,108.009766,108.011720,108.013673,108.015626,108.017579,...,136.982421,136.984374,136.986327,136.988280,136.990234,136.992187,136.994140,136.996093,136.998047,137.000000
0,-82.5,-76.5,-75.0,-97.5,-96.0,-89.0,-84.0,-82.5,-86.0,-87.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
1,-78.5,-80.5,-83.0,-92.0,-96.5,-99.0,-97.0,-94.5,-95.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
2,-83.0,-87.0,-90.5,-91.0,-87.5,-91.0,-90.5,-90.5,-91.0,-92.5,...,-101.0,-101.0,-101.0,-101.0,-99.5,-99.5,-105.0,-101.0,-101.0,-101.0
3,-72.5,-74.0,-80.5,-72.5,-69.5,-68.5,-71.0,-78.5,-81.5,-82.5,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
4,-75.5,-75.0,-74.5,-78.5,-81.5,-82.5,-78.0,-79.5,-82.5,-82.5,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16238,-90.5,-93.0,-99.5,-93.5,-94.0,-95.5,-98.0,-101.0,-101.0,-95.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
16239,-85.0,-87.5,-91.0,-92.0,-92.5,-91.5,-92.5,-94.0,-100.0,-93.5,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
16240,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-96.5,-90.5,...,-101.0,-101.0,-101.0,-99.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
16241,-101.0,-101.0,-101.0,-99.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-98.5,-109.5,-101.0,-101.0,-101.0


In [ ]:
identify_peaks(df, occ=10)

  0%|          | 0/14848 [00:00<?, ?it/s]

[(108.0, '19.57%'),
 (108.00195325655014, '16.34%'),
 (108.00585976965043, '13.42%'),
 (108.00390651310029, '12.98%'),
 (108.00781302620058, '10.46%')]

In [ ]:
blocos = [(k, len(v)) for k,v in cpv['blocks'].items()]
blocos.sort(key=lambda x: x[1], reverse=True)
blocos

[((68, 51), 11962),
 ((40, 1), 8993),
 ((68, 91), 462),
 ((68, 61), 372),
 ((68, 81), 219),
 ((42, 1), 149),
 ((68, 71), 144),
 ((68, 121), 92),
 ((42, 0), 2),
 ((21, 0), 1),
 ((42, 51), 1),
 ((42, 61), 1),
 ((42, 71), 1),
 ((42, 81), 1),
 ((42, 91), 1),
 ((42, 101), 1),
 ((42, 111), 1),
 ((42, 121), 1),
 ((42, 131), 1),
 ((42, 141), 1),
 ((68, 301), 0)]

In [ ]:
%%time
c = cpv['blocks'][(68,51)]
df = extract_levels(c)
df

Wall time: 481 ms


,108.000000,108.001953,108.003907,108.005860,108.007813,108.009766,108.011720,108.013673,108.015626,108.017579,...,136.982421,136.984374,136.986327,136.988280,136.990234,136.992187,136.994140,136.996093,136.998047,137.000000
0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
1,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
2,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
3,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
4,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11957,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
11958,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
11959,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0
11960,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,...,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0,-101.0


In [ ]:
identify_peaks(df, occ=10)

  0%|          | 0/14848 [00:00<?, ?it/s]

[(133.99979793897757, '15.32%'),
 (120.44810399407288, '14.12%'),
 (120.45005725062302, '14.02%'),
 (131.55041422509598, '12.87%'),
 (131.54846096854584, '12.79%'),
 (121.34855526368963, '11.72%'),
 (134.00175119552773, '11.54%'),
 (133.99784468242743, '11.52%'),
 (121.35050852023977, '11.17%')]